## Trabalho II - Computação Gráfica
### Membros do grupo:
- Beatriz Aparecida Diniz, 11925430
- João Lucas Rodrigues Constantino, 11795763
- Leandro Sena, 9293060
- Tulio Santana, 11795526

In [31]:
# Para instalar a API PyWaveFront
# Necessário para carregar os modelos
# %pip install pywavefront

# Inicialização da Janela

In [32]:
# Dependências gerais
import math
from functools import partial

# Dependências locais
from utils.glfw_window import GLFWWindow
from utils.opengl_session import OpenGLSession
from utils.camera import RestrictedCamera
from utils.components import *
from utils.transformations import *

# Propriedades da janela
WINDOW_WIDTH = 1280
WINDOW_HEIGHT = 720
WINDOW_TITLE = "Trabalho 2 - Computação Gráfica"

# Inicialização da janela e da sessão
window = GLFWWindow(WINDOW_WIDTH, WINDOW_HEIGHT, WINDOW_TITLE)
session = OpenGLSession(window)

# Inicialização da câmera
MIN_XPOS, MIN_YPOS, MIN_ZPOS = -10.0, -10.0, -10.0
MAX_XPOS, MAX_YPOS, MAX_ZPOS = +10.0, +10.0, +10.0
camera = RestrictedCamera (
    speed = 0.1, 
    last_xpos=WINDOW_WIDTH/2, 
    last_ypos=WINDOW_HEIGHT/2, 
    min_xpos=MIN_XPOS, min_ypos=MIN_YPOS, min_zpos=MIN_ZPOS,
    max_xpos=MAX_XPOS, max_ypos=MAX_YPOS, max_zpos=MAX_ZPOS
)

# Construção dos Componentes

In [33]:
# Definição dos Objetos e deslocamentos iniciais na cena

# Modelo da Garota
girl = WaveFrontObject (
    "obj/girl/girl OBJ.obj", 
    model = translate(0.6, 9.8, 0.1, scale(0.1, 0.1, 0.1))
)

# Modelo da casa
cottage = WaveFrontObject (
    "obj/cottage/Cottage_FREE.obj", 
    model = scale(0.1, 0.1, 0.1)
)

# Modelo da Torre
tower = WaveFrontObject (
    "obj/tower/wooden watch tower2.obj",
    model = scale(0.1, 0.1, 0.1)
)

# Modelo da Lua
moon = WaveFrontObject(
    "obj/moon/Moon 2K.obj",
    model = translate(10.0, 0.0, 2.0, scale(0.6, 0.6, 0.6))
)

# Modelo da Mesa
table = WaveFrontObject(
    "obj/table/table.obj", 
    model = translate(-52.0, 235.0, 1.0, scale(0.004, 0.004, 0.007))
)

# Modelo do Penguin
penguin = WaveFrontObject(
    "obj/penguin/PenguinBaseMesh.obj", 
    model = translate(2.0, 0.2, 2.0, scale(0.2, 0.2, 0.2))
)

# Modelo do Gerador
generator = WaveFrontObject(
    "obj/generator/Emergency_Backup_Generator-(Wavefront OBJ).obj",
    model = translate(-7.0, 2.2, 10.0, scale(0.1, 0.1, 0.1))
)

# Modelo da Planta
plant = WaveFrontObject(
    "obj/plant/Low-Poly Plant_.obj",
    model = translate(-3.0, 9.4, -0.75, scale(0.1, 0.1, 0.1))
)

# Modelo do Servidor
server = WaveFrontObject(
    "obj/server/server computer.obj",
    model = yrotate(-45.0, translate(-450.0, 45.0, 500.0, scale(0.001, 0.001, 0.001)))
)

# Modelo do Chão de Madeira
wood_floor = WaveFrontObject(
    "obj/floor/wood-floor.obj",
    model = yrotate(90.0, scale(2.5, 5.0, 2.5))
)

# Modelo da Grama
grass = WaveFrontObject(
    "obj/floor/stone-floor.obj",
    model = translate(0.0, -50.0, 0.0, yrotate(90.0, scale(100.0, 0.0, 100.0)))
)

# Registro das componentes na sessão
session.add_components([
    girl, tower, cottage, moon,
    table, penguin, generator,
    plant, server, wood_floor,
    grass
])

Unimplemented OBJ format statement 's' on line 's 1'


Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's off'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 's' on line 's 1'
Unimplemented OBJ format statement 'Tf' on line '	Tf 1 1 1'
Unimplemented OBJ format statement 'g' on line 'g Box004'
Unimplemented OBJ format statement 's' on line 's 8'
Unimplemented OBJ format statement 's' on line 's 2'
Unimplemented OBJ format statement 's' on line 's 4'
Unimplemented OBJ format statement 's' on line 's 16'
Unimplemented OBJ format statement 's' on line 's 64'
Unimplemented OBJ format state

# Transformações

In [34]:
# Atributos de controle dos modelos
moon_angle = 0.0
girl_angle = 0.0
penguin_angle = 0.0

# Movimento de deslocamento da Lua; a luz se move junto
def transform_moon():

    # Ajuste angular
    global moon_angle
    moon_angle += 1e-3
    if moon_angle > 2.0 * math.pi:
        moon_angle = 0.0
    
    # Parâmetros de posicionamento
    tx = math.cos(moon_angle) * 32
    ty = math.sin(moon_angle) * 32
    tz = 0.0

    # Ajuste das posições
    moon.set_model(translate(tx,ty,tz))
    session.move_light(tx, ty, tz)

# Para transformar a garota
# ACHISMO DO TULIP: talvez seja bacana bindar pra uma tecla só, mas tbm n sei
def transform_girl():
    global girl_angle 
    girl_angle += 1e-5
    if girl_angle > 2.0 * math.pi:
        girl_angle = 0.0
    girl.set_model(yrotate(girl_angle, girl.model))

# Para transformar o penguin
def transform_penguin():
    global penguin_angle
    penguin_angle += 1e-6
    if penguin_angle > 2.0 * math.pi:
        penguin_angle = 0.0
    penguin.set_model(yrotate(penguin_angle, penguin.model))

# Eventos de teclado e de mouse

In [35]:
# Para aumentar ou diminuir a velocidade da câmera
def modify_camera_speed(camera, increase=True):
    original_speed = camera.speed
    if increase == False:
        original_speed *= -1
    def closure(*_):
        camera.speed += original_speed
    return closure

# Atribuição dos eventos de movimentação da câmera
window.add_key_callback("W", camera.move_forward)
window.add_key_callback("D", camera.move_right)
window.add_key_callback("S", camera.move_backward)
window.add_key_callback("A", camera.move_left)
window.add_cursor_callback(camera.move_front)
window.add_key_callback("Q", modify_camera_speed(camera, False))
window.add_key_callback("E", modify_camera_speed(camera, True))

# Controle da projeção
fovy = 45.0
near = 0.1
far = 1000.0

# Controle de fovy de perspectiva
def change_fovy(*_, increase=15.0):
    global fovy
    fovy += increase
    if fovy > 360.0:
        fovy = 0.0

# Controle de near de perspectiva
def change_near(*_, increase=0.1):
    global near
    near += increase

# Controle de far de perspectiva
def change_far(*_, increase=100.0):
    global far
    far += increase

# Registro dos controles de alteração de perspectiva
window.add_key_callback("R", change_fovy)
window.add_key_callback("F", partial(change_fovy, increase=-15.0))
window.add_key_callback("T", change_near)
window.add_key_callback("G", partial(change_near, increase=-0.1))
window.add_key_callback("Y", change_far)
window.add_key_callback("H", partial(change_far, increase=-100.0))

# Execução

In [36]:
# Processamento dos buffers
session.process_buffers()

# Programa principal
for _ in window.display():

    # Limpeza da janela
    session.clear()

    # Transformações
    transform_moon()
    transform_girl()
    transform_penguin()

    # Renderização
    session.render()

    # Visualização
    session.view(camera.view)

    # Projeção
    session.projection(window.get_projection(fovy, near, far))